# AutoTS Example

AutoTS是用于支持PaddleTS的自动机器学习能力组件。

AutoTS 可以支持 PaddleTS 模型和 pipeline 的自动超参数选择，减少人工介入成本，降低专业门槛。

## 1. 准备数据

In [ ]:
from paddlets.datasets.repository import get_dataset
tsdataset = get_dataset("UNI_WTH")

## 2 构造和训练

通过四行代码，我们利用 MLPRegressor 初始化了一个 AutoTS 模型。 AutoTS 会在训练的过程中自动进行超参优化


In [ ]:
from paddlets.models.forecasting import MLPRegressor
from paddlets.automl.autots import AutoTS
autots_model = AutoTS(MLPRegressor, 96, 2)
autots_model.fit(tsdataset, n_trials=5)

## 3. 搜索空间
对于超参数优化来说，你可以定义一个搜索空间。如果你没有指定一个搜索空间，我们也为每个 PaddleTS 模型内置了推荐的默认搜索空间。

你可以利用搜索空间去控制你的超参的取值是如何采样的，控制值范围是多少。

下面是一个指定了搜索空间的 autots pipeline 的例子：

In [ ]:
from ray.tune import uniform, qrandint, choice
from paddlets.transform import Fill

sp = {
    "Fill": {
        "cols": ['WetBulbCelsius'],
        "method": choice(['max', 'min', 'mean', 'median', 'pre', 'next', 'zero']),
        "value": uniform(0.1, 0.9),
        "window_size": qrandint(20, 50, q=1)
    },
    "MLPRegressor": {
        "batch_size": qrandint(16, 64, q=16),
        "use_bn": choice([True, False]),
        "max_epochs": qrandint(10, 50, q=10)
    }
}
autots_model = AutoTS([Fill, MLPRegressor], 25, 2, search_space=sp, sampling_stride=25)
autots_model.fit(tsdataset, n_trials=1)
sp = autots_model.search_space()
predicted = autots_model.predict(tsdataset)

搜索空间定义的 API 可以参考：https://docs.ray.io/en/latest/tune/api_docs/search_space.html

## 4. 搜索算法

PaddleTS 的搜索算法是利用多个开源的超参优化库包装而成。

我们内置了如下几种搜索算法：
[“Random”, “CMAES”, “TPE”, “CFO”, “BlendSearch”, “Bayes”]

对于这些超参优化库的细节，可以参考它们对应的开源文档。

你可以通过下面的方式指定搜索算法：



In [ ]:
autots_model = AutoTS(MLPRegressor, 96, 2, search_alg="CMAES")

如果搜索算法没有被指定，我们会默认使用TPE算法。